In [ ]:
!pip install s3fs

In [ ]:
import boto3
from botocore.exceptions import ClientError
import configparser
import json
import pandas as pd

config = configparser.ConfigParser()
config.read('aws.cfg')

aws_access_key = config['AWS']['aws_access_key_id']
aws_secret_key = config['AWS']['aws_secret_access_key']


In [ ]:
# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1", aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)

# Set the model ID
model_id = "amazon.titan-text-express-v1:0"

In [ ]:
# Read external landmark data from S3
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

landmarks = s3.download_file('capstone-techcatalyst-conformed', 'group1/landmark_data/part-00000-tid-8270335314995015129-7c07741e-3c51-4b23-9a75-95c6c641450b-65-1-c000.csv', 'part-00000-tid-8270335314995015129-7c07741e-3c51-4b23-9a75-95c6c641450b-65-1-c000.csv')
landmark_df = pd.read_csv('/workspaces/thePANDAsCapstone/Pipeline Stages/Databricks/part-00000-tid-8270335314995015129-7c07741e-3c51-4b23-9a75-95c6c641450b-65-1-c000.csv')

In [ ]:
# Build dictionary which maps the landmark name to the landmark description to be used with Bedrock
landmark_list = landmark_df['Landmark_Name'].to_list()
description_list = landmark_df['USE_ORIG'].to_list()
build_type = landmark_df['BuildType'].to_list()
landmark_dict = {landmark_list[i]: description_list[i] for i in range(len(landmark_list))}


In [ ]:
#DO NOT RERUN
# Collects all the landmark names categorized
categories = {}


In [ ]:
# Function that calls Bedrock client by taking in query, running it through the model, and returns the output string

def bedrock(query):
    user_message = query
    conversation = [
        {
            "role": "user",
            "content": [{"text": user_message}],
        }
    ]

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId="amazon.titan-text-express-v1",
            messages=conversation,
            inferenceConfig={"maxTokens":4096,"stopSequences":["User:"],"temperature":0,"topP":1},
            additionalModelRequestFields={}
        )


        # Extract and print the response text.
        response_text = response["output"]["message"]["content"][0]["text"]
        print(response_text)

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

    return response_text

In [ ]:
# Function that provides the query to the Bedrock function and parses the returned string in JSON format to extract the category of the landmark

def categorize(start, end, categories):
    print(f'rows {start} - {end}')
    query = f""" {dict(list(landmark_dict.items())[start:end])}

    Assign one of each one of these names to one of these categories using the description, choosing from: Architecture, Religion, Transportation, Education, Civic, Entertainment, Financial, Commercial, Industrial, Historic. Leave empty if none of the categories match. Return output in JSON format with keys "Name" and "Category". Return only one category for every name with nothing in the output except the JSON object.

    Use the following descriptions for the categories:
    Architecture: infrastructure, structure
    Religion: churches, religious
    Transportation: airports, railroads, bridges
    Education: school, university
    Civic: government, military
    Entertainment: arts, sports, theaters, museum, recreation, amusement
    Financial: banks
    Commercial: businesses, shopping, hotel
    Industrial: factories
    Historic: cemetery, relic


"""
    response_text = bedrock(query)


    pos = response_text.find('{')
    pos2 = response_text.find('[')

    if response_text[pos] == '{':
        json_output = json.loads(response_text[pos:-3])
        obj = json_output['rows']
    elif response_text[pos2] == '[':
        json_output = json.loads(response_text[pos2:-3])
        obj = json_output



    names = [x['Name'] for x in obj] 
    category = [x['Category'] for x in obj] 

    categorized = {names[i]: category[i] for i in range(len(names))}

    categories.update(categorized)
    return categorized


In [ ]:
# Loops through the landmarks 10 at a time

for i in range(10, len(landmark_dict.keys()), 10):    
    categorize(i-10, i, categories)


In [ ]:
# Validation that all landmarks in original landmark dictionary have been categorizezd
diff = landmark_dict.keys() - categories.keys()

remaining = {}
for i, x in enumerate(list(landmark_dict.keys())):
    if x in diff:
        remaining[x] = i

remaining

In [ ]:
categories

In [ ]:
# Convert to pandas dataframe
categories_df = pd.DataFrame({'Name': categories.keys(), 'Category': categories.values()})

In [ ]:
# Initial cleaning
name = categories_df['Name'].drop_duplicates()
categories_df['Category'] = categories_df['Category'].apply(lambda x: x.split(',')[0])

In [ ]:
categories_df['Category'].value_counts()

In [ ]:
# Consolidation of categories

categories_df['Category'] = categories_df['Category'].replace(to_replace=['Mixed-use','Office Building', 'Hotel'], value='Commercial')
categories_df['Category'] = categories_df['Category'].replace(to_replace=['Educational', 'Education'], value='Civic')
categories_df['Category'] = categories_df['Category'].replace(to_replace=['Religious'], value='Religion')
categories_df['Category'] = categories_df['Category'].replace(to_replace=['Bank'], value='Financial')
categories_df['Category'] = categories_df['Category'].replace(to_replace=['Movie Theater', 'Amusement', 'Recreational'], value='Entertainment')
categories_df['Category'] = categories_df['Category'].replace(to_replace=['Mausoleum', 'Cemetery'], value='Historic')
categories_df['Category'] = categories_df['Category'].replace(to_replace=['Infrastructure'], value='Industrial')



In [ ]:
categories_df['Category'].value_counts()

In [ ]:
categories_df

In [ ]:
# Write dataframe to S3

categorized_path = 's3://capstone-techcatalyst-conformed/group1/categorized/categorized_landmarks.csv'
categories_df.to_csv(categorized_path, header = 'True', storage_options={
                   'key' : aws_access_key,
                   'secret' : aws_secret_key
               })